# Sage Shaw

### ME 571 - Spring 2018

# HW 1 - Problem 4

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import random
import multiprocessing as mp
from timeit import timeit

N = 10**7

## Single Processor w/o Numpy

In [11]:
nums = [(random(),random()) for i in range(N)]

def single_calc():
    #r = [x**2 + y**2 for x,y in nums]
    #hits = [1> radius for radius in r]
    #return 4 * sum(hits)/len(nums)
    return 4 * sum([x**2 + y**2 <= 1 for x, y in nums]) /N
print(single_calc())

setup = """
from random import random
N = """ + str(N) + """
nums = [(random(),random()) for i in range(N)]
def single_calc():
    return 4 * sum([x**2 + y**2 <= 1 for x, y in nums]) /N
"""

3.1407896


In [12]:
%%timeit
single_calc()

3.41 s ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#%%timeit
#print( 4 * sum([x**2 + y**2 <= 1 for x, y in nums]) /N )
repeats = 10
single = timeit(stmt="single_calc()", setup=setup, number=repeats)
print(single/repeats)

## Single Processor w/ Numpy

In [37]:
num = np.random.rand(N,2)

In [38]:
%%timeit
r = num[:,0]**2 + num[:,1]**2
hits = np.greater(1, r)
pi = 4 * np.sum(hits)/ N

102 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Multiprocessor w/o Numpy

In [15]:
#nums = [(random(),random()) for i in range(N)]

def calc_pi(proc_num, sublist_size, ret_queue, nums):
    #sublist_size = len(nums)//p
    my_nums = nums[proc_num*sublist_size : min((proc_num+1)*sublist_size, len(nums))]
    pi = 4 * sum([x**2 + y**2 <= 1 for x, y in my_nums]) / len(my_nums)
    ret_queue.put(pi)
    return

In [10]:
def multi_calc(nums, p):
    manager = mp.Manager()
    ret_queue = mp.Queue()
    sublist_size = len(nums)//p
    #sublists = [nums[i:i+sublist_size] for i in range(0, len(nums), sublist_size)]
    #jobs = [mp.Process(target=calc_pi, args=(ret_queue, sublist) ) for sublist in sublists]
    jobs = [mp.Process(target=calc_pi, args=(i, sublist_size, ret_queue, nums) ) for i in range(p)]
    #for job in jobs:
    #    job.start()
        
    [job.start() for job in jobs]
    [job.join() for job in jobs]
    #for job in jobs:
     #   job.join()
        
    pi = 0
    while not ret_queue.empty():
        temp = ret_queue.get()
        pi += temp
    return pi/p

In [11]:
nums = [(random(),random()) for i in range(N)]
#nums = np.random.random( (N,2))

In [18]:
%%timeit
pi = multi_calc(nums, 1)

4.26 s ± 39.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
pi = multi_calc(nums, 2)

2.31 s ± 32.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
pi = multi_calc(nums, 3)

1.62 s ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
pi = multi_calc(nums, 4)

1.34 s ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
pi = multi_calc(nums, 5)

1.44 s ± 59.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
pi = multi_calc(nums, 6)

1.42 s ± 50.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
pi = multi_calc(nums, 8)

1.38 s ± 24.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
pi = multi_calc(nums, 16)

1.49 s ± 12.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
pi = multi_calc(nums, 32)

1.71 s ± 47.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Multiprocessor w/ Numpy

In [14]:
nums = np.random.random( (N,2) )

def calc_pi(proc_num, sublist_size, ret_queue, nums):
    #sublist_size = len(nums)//p
    my_nums = nums[proc_num*sublist_size : min((proc_num+1)*sublist_size, len(nums))]
    x,y = my_nums[:,0], my_nums[:,1]
    pi = 4 * sum( np.greater(1, x**2 + y**2))/len(my_nums)
    #pi = 4 * sum([x**2 + y**2 <= 1 for x, y in my_nums]) / len(my_nums)
    #r = x**2 + y**2
    #hits = np.greater(1,r)
    #pi = 4 * sum(hits)
    ret_queue.put(pi)
    return

In [13]:
%%timeit
pi = multi_calc(nums, 1)

49.2 s ± 367 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
pi = multi_calc(nums, 4)

8.39 s ± 28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Multiprocessor w/Numpy - Pythonic

In [7]:
nums = np.random.random( (N,2) )

def calc_pi(proc_num, sublist_size, ret_queue, nums):
    #sublist_size = len(nums)//p
    my_nums = nums[proc_num*sublist_size : min((proc_num+1)*sublist_size, len(nums))]
    x,y = my_nums[:,0], my_nums[:,1]
    pi = 4 * sum( np.greater(1, x**2 + y**2))/len(my_nums)
    #pi = 4 * sum([x**2 + y**2 <= 1 for x, y in my_nums]) / len(my_nums)
    #r = x**2 + y**2
    #hits = np.greater(1,r)
    #pi = 4 * sum(hits)
    ret_queue.put(pi)
    return

## Analysis
By far the best method is to use **Numpy** without the **multiprocessing** module. **Numpy** is very well optimized and uses BLAS and multithreading via lower level languages (complied languages) to optimize runtimes. Interestingly using the **multiprocessing** module with Numpy drastically increases runtime and appears to take 8-10 times longer than the **multiprocessing** module with list comprehensions (the most efficient way to work with lists in Python 3). 

For the parallelized runs, we did indeed see an increase in speed when more processes were used. Using one process introduced extra overhead and took longer, but as the number of processors rose the time decreased. This capped at 4 processes, which is expected since the machine has 4 processor cores and cannot physically run more than 4 processes at a time.

## A note on Pythonic code
In the math portion of the **calc_pi** functions above I experimented using multiple lines of clean code (i.e. Pythonic code) vs compressed one-line versions. In the plain-python version the one-line version seemed to be slightly faster. I believe this is due to the fact that it only generates a single list, whereas the cleaner version stores the outputs in separate lists. It is somewhat unfortunate because the purpose of the python programming language is to create simple, human-readable code.